In [1]:
# Подключим необходимые библиотеки
import numpy as np
import pandas as pd
import scipy as scipy
from copy import deepcopy
import math
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from scipy.stats import t, chi2, f, norm, poisson, binom, uniform, expon, logistic
import seaborn
import networkx as nx
from sklearn.model_selection import train_test_split

ATE, Средний эффект воздействия selfdom на income
serfdom - крепостное  право

income — выручка крестьянина за месяц в рублях (с учетом выплат помещику), 
используемая в качестве меры производительность труда.


• age — возраст в года

 
• male — пол (0 — женщина, 1 — мужчин
 
• educ — факт наличия школьного образования (0 — не учился в воскресной школе, 
— учился в воскрей школе
)
. 
• married — брак (0 — не состоит в браке, 1 — состоит в б

ке). 
• children — число

етей. 
• serfdom — статус крепостного (0 — не крепостной, 1 — креп

тной). 
• weight — вес крестьянина в кил

раммах. 
• height — рост крестьянина

 метрах. 
• income — выручка крестьянина за месяц в рублях (с учетом выплат пмещику), 
используемая в качестве меры производитель

сть труда. 
• progress — наличие у помещика прогрессивных взглядов на свободу лчности (0 —
 отсутствуют,

 — имеются). 
• soil — плодородность земли в регионе проживания крестьянина (0  низкая, 1 — 
средня,— высокая). а .

# ATE

In [3]:
df = pd.read_excel('Турнир 1 train.xlsx')   
df.head()

,age,male,educ,weight,height,children,soil,income,serfdom,progress
0,24,1,1,76,1.74,4,1,101,0,0
1,18,1,0,51,1.41,2,0,64,0,0
2,36,1,0,73,1.78,7,0,90,1,0
3,29,1,0,88,1.74,5,2,81,1,1
4,34,1,0,67,1.62,7,1,113,1,0


## Наивная оценка как разница в выборочных средних

In [5]:
ATE_naive = np.mean(df.income[df['serfdom'] == 1]) - np.mean(df.income[df['serfdom'] == 0])

# Сравнение точного приближения и наивной оценки
print(pd.DataFrame(data    = [ATE_naive],
                   index   = ['ATE naive'],
                   columns = ['Оценка']))

             Оценка
ATE naive  7.610162


## OLS

In [23]:
# Оценим средний эффект воздействия с помощью МНК

# МНК оценивание уравнения цены за квадратный метр для
# квартир, рядом с которыми нет метро
import statsmodels.api as sm
y0  = df.loc[df.serfdom == 0, 'income']
x0  = df.loc[df.serfdom == 0, df.columns.drop(['income',  'serfdom'])]
x0  = sm.add_constant(x0)
ls0 = sm.OLS(y0, x0).fit()

# МНК оценивание уравнения цены на квадратный метр для
# квартир, рядом с которыми есть метро
y1  = df.loc[df.serfdom == 1, ['income']]
x1  = df.loc[df.serfdom == 1, df.columns.drop(['income',  'serfdom'])]
x1  = sm.add_constant(x1)
ls1 = sm.OLS(y1, x1).fit()

# Оценим цены за квадратный метр при наличии и отсутствия метро
# с помощью полученных МНК оценок
x = df.loc[:, df.columns.drop(['income', 'serfdom'])]
x = sm.add_constant(x)
y = df.loc[:, 'income']

In [25]:
  # МНК оценка E(income0 | X) 
income0_ls = ls0.predict(x)
  # МНК оценка E(income1 | X) 
income1_ls = ls1.predict(x)

# Оценки CATE
CATE_ls = np.array(income1_ls - income0_ls)

# Оценка ATE как средняя разница в прогнозах МНК оценок
ATE_ls = np.mean(CATE_ls)

# Сравним результаты
print(pd.DataFrame(data    = [ATE_naive, ATE_ls],
                   index   = ['ATE naive', "ATE ls"],
                   columns = ['Оценка']))

             Оценка
ATE naive  7.610162
ATE ls    -4.276760


## S-learner

In [35]:
# Оценивание ATE и CATE с помощью S-learner
from sklearn.ensemble import GradientBoostingRegressor
np.random.seed(123)
# Подготовим данные
y = df.loc[:, ['income']]
x = df.loc[:, df.columns.drop(['income'])]

GB2 = GradientBoostingRegressor(random_state=0)
GB2 = GradientBoostingRegressor(learning_rate       = 0.1,
                            min_samples_split       = 6,
                            min_samples_leaf        = 2,
                            max_depth               = 4,
                            max_features            = None,
                            subsample               = 0.75)

GB2.fit(x,y)

# Оценки E(income | X, serfdom = 0) для всех крепостных крестьян (serfdom = 1).
x0= deepcopy(x)
x0["serfdom"] = 0
income0_GB2  = GB2.predict(x0)

# Оценки E(income | X, serfdom = 1) для некрепостных крестьян (serfdom = 0).
x1         = deepcopy(x)
x1["serfdom"] = 1
income1_GB2  = GB2.predict(x1)

# Оценки CATE
CATE_S = income1_GB2 - income0_GB2

# Оценка ATE
ATE_S = np.mean(CATE_S)

C:\Users\annaz\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


In [37]:
# Сопоставим результаты
print(pd.DataFrame(data    = [ATE_naive, ATE_ls,
                              ATE_S],
                   index   = ['ATE naive', 'ATE ls',
                              'ATE S-learner'],
                   columns = ['Оценка']))

                 Оценка
ATE naive      7.610162
ATE ls        -4.276760
ATE S-learner -3.056613


## T-learner

In [43]:
np.random.seed(123)
import statsmodels.api as sm
y0  = df.loc[df.serfdom == 0, 'income']
x0  = df.loc[df.serfdom == 0, df.columns.drop(['income',  'serfdom'])]
x0  = sm.add_constant(x0)
ls0 = sm.OLS(y0, x0).fit()

# МНК оценивание уравнения цены на квадратный метр для
# квартир, рядом с которыми есть метро
y1  = df.loc[df.serfdom == 1, ['income']]
x1  = df.loc[df.serfdom == 1, df.columns.drop(['income',  'serfdom'])]
x1  = sm.add_constant(x1)
ls1 = sm.OLS(y1, x1).fit()

x = df.loc[:, df.columns.drop(['income', 'serfdom'])]
x = sm.add_constant(x)
y = df.loc[:, 'income']

# Оценивание ATE и CATE с помощью T-learner
from sklearn.ensemble import GradientBoostingRegressor

# rf = RandomForestClassifier(max_depth=5, 
#                              max_features=7, min_samples_leaf=1, min_samples_split=2, 
#                              n_estimators=100
#                             )
# rf.fit(features, target)
# # Обучение оценивать E(income | X, serfdom = 0)
# rf.fit(x0,y0)
# # Оценки E(income | X, serfdom = 0) для крепостных крестьях (serfdom = 1).
# income0_rf = rf.predict(x)
# # Обучение оценивать E(price | X, subway = 1)
# rf.fit(x1, y1)
# # Оценки E(income | X, serfdom = 1) для некрепостных крестьян (serfdom = 0).
# income1_rf = rf.predict(x)
# # # Оценки CATE
# CATE_T = income1_rf - income0_rf


GB = GradientBoostingRegressor(learning_rate       = 0.1,
                            min_samples_split       = 6,
                            min_samples_leaf        = 2,
                            max_depth               = 4,
                            max_features            = None,
                            subsample               = 0.75)
# # Обучение оценивать E(income | X, serfdom = 0)
GB.fit(x0,y0)
# Оценки E(income | X, serfdom = 0) для крепостных крестьях (serfdom = 1).
income0_GB = GB.predict(x)
# Обучение оценивать E(price | X, subway = 1)
GB.fit(x1, y1)
# Оценки E(income | X, serfdom = 1) для некрепостных крестьян (serfdom = 0).
income1_GB = GB.predict(x)

# # Оценки CATE
CATE_T = income1_GB - income0_GB

# Оценка ATE
ATE_T = np.mean(CATE_T)

C:\Users\annaz\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


In [45]:
# Сопоставим результаты
print(pd.DataFrame(data    = [ATE_naive, ATE_ls,
                              ATE_S, ATE_T],
                   index   = ['ATE naive', 'ATE ls',
                              'ATE S-learner', 'ATE T-learner', ],
                   columns = ['Оценка']))

                 Оценка
ATE naive      7.610162
ATE ls        -4.276760
ATE S-learner -3.056613
ATE T-learner -3.275184


## Оценивание с помощью обратного взвешивания на вероятности IPW

In [47]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors  import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

np.random.seed(123)

#Подготовим данные
target   = df.loc[:, ['serfdom']]
features = df.loc[:, df.columns.drop(['income', 'serfdom'])]

gb = GradientBoostingClassifier(learning_rate=0.1, max_depth=5)
gb.fit(features, target)
prob = gb.predict_proba(features)[:, 1]
prob = np.clip(prob, 1e-2, 1 - 1e-2)  # Избегаем деления на 0


# rf3 = RandomForestClassifier(max_depth=5, 
#                              max_features=7, min_samples_leaf=1, min_samples_split=2, 
#                              n_estimators=100
#                             )
# rf3.fit(features, target)
# prob = rf3.predict_proba(features)[:, 1]
# # Убедимся, что вероятности не равны 0 или 1
# prob = np.clip(prob, 1e-5, 1 - 1e-5)  # Избегаем деления на 0


# Оценим псевдоисходы
income_pseudo = df['income'] * (df['serfdom'] / prob -  (1 - df['serfdom']) / (1 - prob))

# Оценим ATE
ATE_IPW = np.mean(income_pseudo)

C:\Users\annaz\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [49]:
print(max(income_pseudo))
min(income_pseudo)

2930.9745879552843


-2689.892956750197

In [51]:
# Сопоставим результаты
print(pd.DataFrame(data    = [ATE_naive, ATE_ls,
                              ATE_S, ATE_T, ATE_IPW],
                   index   = ['ATE naive', 'ATE ls',
                              'ATE S-learner', 'ATE T-learner', 'ATE_IPW'],
                   columns = ['Оценка']))

                 Оценка
ATE naive      7.610162
ATE ls        -4.276760
ATE S-learner -3.056613
ATE T-learner -3.275184
ATE_IPW       -6.840156


## Метод двойной устойчивости

In [57]:
# Оценим ATE с помощью DR метода с двойной устойчивостью
ATE_DR = np.mean((income1_GB - income0_GB) + \
                 df['serfdom'] * (df['income'] - income1_GB) / prob - \
                 (1 - df['serfdom']) * (df['income'] - income0_GB) / (1 - prob))

In [59]:
# Сопоставим результаты
print(pd.DataFrame(data    = [ATE_naive, ATE_ls,
                              ATE_T, ATE_S, ATE_IPW, ATE_DR],
                   index   = ['ATE naive', 'ATE ls',
                              'ATE T-learner', 'ATE S-learner',
                              'ATE IPW', 'ATE DR'],
                   columns = ['Оценка']))

                 Оценка
ATE naive      7.610162
ATE ls        -4.276760
ATE T-learner -3.275184
ATE S-learner -3.056613
ATE IPW       -6.840156
ATE DR        -3.401915


## Double ML

In [61]:
df.columns

Index(['age', 'male', 'educ', 'weight', 'height', 'children', 'soil', 'income',
       'serfdom', 'progress'],
      dtype='object')

In [63]:
#!pip install doubleml
import doubleml as dml
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
np.random.seed(125)

dml_standard_data = dml.DoubleMLData(
                            data   = df,
                            y_col  = 'income',
                            d_cols = 'serfdom',
                            x_cols = ['age', 'male', 'educ', 'weight', 'height', 'children', 'soil', 'progress'
                                     ])

# Метод оценивания E(Y | X, T)
g_Y = GradientBoostingRegressor(learning_rate       = 0.1,
                            min_samples_split       = 6,
                            min_samples_leaf        = 2,
                            max_depth               = 4,
                            max_features            = None,
                            subsample               = 0.75)

# Метод оценивания E(T | X)
g_T = GradientBoostingClassifier(learning_rate=0.1, max_depth=5)

# Подготовка объекта
dml_standard = dml.DoubleMLIRM(obj_dml_data = dml_standard_data,
                               ml_g         = g_Y,
                               ml_m         = g_T,
                               n_rep        = 1,
                               n_folds      = 5)

# Оценим параметры
dml_standard.fit()

# Сохраним оценку
ATE_dml_standard = dml_standard.coef[0]

In [65]:
# Сопоставим результаты
print(pd.DataFrame(data    = [ATE_naive, ATE_ls,
                              ATE_T, ATE_S, ATE_IPW, ATE_DR, ATE_dml_standard],
                   index   = ['ATE naive', 'ATE ls',
                              'ATE T-learner', 'ATE S-learner',
                              'ATE IPW', 'ATE DR', 'ATE_dml_standard'],
                   columns = ['Оценка']))

                    Оценка
ATE naive         7.610162
ATE ls           -4.276760
ATE T-learner    -3.275184
ATE S-learner    -3.056613
ATE IPW          -6.840156
ATE DR           -3.401915
ATE_dml_standard -3.941823


In [67]:
## Опираясь на полученные оценки, хочется верить, что истинное значение АТЕ где-то в диапазоне (-4;-3)
## Больше всего доверие вызывает метод двойной устойчивости и Double ML
print('final ATE:  ', ATE_dml_standard)

final ATE:   -3.941823216928635


# Income

In [105]:
y   = df.loc[:, ['income']]
x = df.loc[:, df.columns.drop(['income'])]
y   = np.squeeze(y)                # преобразуем из вектора столбца в одномерный массив
x.head()

# Разделим выборку на обучающую и тестовую
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.2, random_state = 777)

# Убедимся, что обучающая и тестовая выборки имеют верные пропорции
print(x_train.index.size, x_test.index.size) # признаки
print(y_train.index.size, y_test.index.size)     # целевая переменная
x_train.head()

8000 2000
8000 2000


,age,male,educ,weight,height,children,soil,serfdom,progress
8490,18,0,1,73,1.69,1,1,0,0
9788,17,0,0,59,1.47,2,0,0,0
5967,28,1,0,91,1.97,6,1,1,0
7519,20,1,0,93,1.80,7,1,1,0
8614,49,0,0,65,1.75,7,1,1,0


## GBoost

In [107]:
from sklearn.ensemble import GradientBoostingRegressor

GB_reg = GradientBoostingRegressor(random_state=0)
GB_reg.fit(x_train, y_train)

print('train:  ', GB_reg.score(x_train, y_train))
print('test:  ', GB_reg.score( x_test, y_test))
pred_GB_reg = GB_reg.predict(x_test)
pred_GB_reg

train:   0.8025507512519634
test:   0.8053270117577189


array([37.2033245 , 59.27461165, 79.59724796, ..., 31.44121117,
       25.74921708, 94.39052869])

In [109]:
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score

## Посчитаем RMSE и MAPE на тренирововчной и тестовой выборках.
MSE_train_GB = mean_squared_error(y_train,  GB_reg.predict(x_train))
MAPE_train_GB = mean_absolute_percentage_error(y_train,  GB_reg.predict(x_train))
MSE_test_GB = mean_squared_error(y_test, pred_GB_reg)
MAPE_test_GB = mean_absolute_percentage_error(y_test, pred_GB_reg)

# Посчитаем RMSE и MAPE с помощью кросс-валидации (чем меньше, тем лучше)
MSE_train_GB_cv  = -np.mean(cross_val_score(GB_reg, x_train, y_train, scoring='neg_mean_squared_error', cv = 5))
MSE_test_GB_cv = -np.mean(cross_val_score(GB_reg, x_test, y_test, scoring='neg_mean_squared_error', cv = 5))
MAPE_train_GB_cv  = -np.mean(cross_val_score(GB_reg, x_train, y_train, scoring='neg_mean_absolute_percentage_error', cv = 5))
MAPE_test_GB_cv = -np.mean(cross_val_score(GB_reg, x_test, y_test, scoring='neg_mean_absolute_percentage_error', cv = 5))

## Выводим все метрики до подбора пораметров
print(pd.DataFrame(data    = {'MSE': [
                              MSE_train_GB,
                              MSE_test_GB,
                              MSE_train_GB_cv
                              ],
                              'MAPE' : [
                              MAPE_train_GB,
                              MAPE_test_GB,
                              MAPE_train_GB_cv
                              ]},
                   index   = ['Gradient Boosting Regression  (train)',
                              'Gradient Boosting Regression  (test)',
                              'Gradient Boosting Regression  (cv, train)'
                              ],
                   columns = ['MSE', 'MAPE']))

                                                 MSE      MAPE
Gradient Boosting Regression  (train)      90.471361  0.115596
Gradient Boosting Regression  (test)       88.165451  0.115675
Gradient Boosting Regression  (cv, train)  97.162927  0.119539


In [111]:
from sklearn.model_selection import GridSearchCV

# Перебираемые значения гиперпараметров
hyperparameters = {
    "learning_rate": [0.1, 0.3, 0.5],
    "min_samples_split": [2, 4, 6],
    "min_samples_leaf": [1, 2, 3],
    "max_depth":[2, 3, 4, 5, 7,],
    "max_features":["sqrt", None],
    "subsample":[0.5, 0.75]
    }

# Перебор гиперпараметров с помощью кросс-валидации
GSCV_GB_reg = GridSearchCV(estimator  = GB_reg,                               # модель
                        param_grid = hyperparameters,                         # перебираемые гиперпараметры
                        scoring    = "neg_mean_squared_error",                # метрика качества модели
                        cv         = 5)                                       # число фолдов в кросс-валидации
GSCV_GB_reg.fit(x_train, y_train)                                             # подбор гиперпараметров на конкретных данных

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=0),
             param_grid={'learning_rate': [0.1, 0.3, 0.5],
                         'max_depth': [2, 3, 4, 5, 7],
                         'max_features': ['sqrt', None],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 4, 6],
                         'subsample': [0.5, 0.75]},
             scoring='neg_mean_squared_error')

In [113]:
# Достанем гиперпараметры, соответствующие лучшей модели
hyperparameters_GB_reg = GSCV_GB_reg.best_params_
print(hyperparameters_GB_reg)

{'learning_rate': 0.1, 'max_depth': 4, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'subsample': 0.75}


In [115]:
# Обучим модель с лучшими гиперпараметрами
GB_reg2 = GradientBoostingRegressor(learning_rate   = hyperparameters_GB_reg["learning_rate"],
                            min_samples_split       = hyperparameters_GB_reg["min_samples_split"],
                            min_samples_leaf        = hyperparameters_GB_reg["min_samples_leaf"],
                            max_depth               = hyperparameters_GB_reg["max_depth"],
                            max_features            = hyperparameters_GB_reg["max_features"],
                            subsample               = hyperparameters_GB_reg["subsample"])
GB_reg2.fit(x_train, y_train)

GradientBoostingRegressor(max_depth=4, min_samples_leaf=3, subsample=0.75)

In [77]:
# # Обучим модель с лучшими гиперпараметрами
# GB_reg2 = GradientBoostingRegressor(learning_rate   = 0.1,
#                             min_samples_split       = 2,
#                             min_samples_leaf        = 3,
#                             max_depth               = 4,
#                             max_features            = None,
#                             subsample               = 0.75)
# GB_reg2.fit(x_train, y_train)

GradientBoostingRegressor(max_depth=4, min_samples_leaf=3, subsample=0.75)

In [117]:
# Оценим точность модели с подобранными гиперпараметрами
MSE_train_GB2 = mean_squared_error(y_train,  GB_reg2.predict(x_train)) # обучающая выборка
MSE_test_GB2 = mean_squared_error(y_test,  GB_reg2.predict(x_test)) # тестовая выборка
MAPE_train_GB2 = mean_absolute_percentage_error(y_train,  GB_reg2.predict(x_train)) # обучающая выборка
MAPE_test_GB2 = mean_absolute_percentage_error(y_test,  GB_reg2.predict(x_test)) # тестовая выборка


# Сравним результаты кросс-валидации (чем меньше, тем лучше)
MSE_train_GB2_cv  = -np.mean(cross_val_score(GB_reg2, x_train, y_train, scoring='neg_mean_squared_error', cv = 5))
MSE_test_GB2_cv = -np.mean(cross_val_score(GB_reg2, x_test, y_test, scoring='neg_mean_squared_error', cv = 5))
MAPE_train_GB2_cv  = -np.mean(cross_val_score(GB_reg2, x_train, y_train, scoring='neg_mean_absolute_percentage_error', cv = 5))
MAPE_test_GB2_cv = -np.mean(cross_val_score(GB_reg2, x_test, y_test, scoring='neg_mean_absolute_percentage_error', cv = 5))

## Выводим RMSE и MAPE метрики до подбора пораметров и после
print(pd.DataFrame(data    = {'MSE': [
                              MSE_train_GB,
                              MSE_test_GB,
                              MSE_train_GB_cv,
                              MSE_train_GB2,
                              MSE_test_GB2,
                              MSE_train_GB2_cv
                              ],
                              'MAPE' : [
                              MAPE_train_GB,
                              MAPE_test_GB,
                              MAPE_train_GB_cv,
                              MAPE_train_GB2,
                              MAPE_test_GB2,
                              MAPE_train_GB2_cv
                              ]},
                   index   = ['Gradient Boosting Regression  (train)',
                              'Gradient Boosting Regression   (test)',
                              'Gradient Boosting Regression    (cv, train)',
                              'Gradient Boosting Regression  2 (train)',
                              'Gradient Boosting Regression  2 (test)',
                              'Gradient Boosting Regression  2 (cv, train)'
                              ],
                   columns = ['MSE', 'MAPE']))

                                                   MSE      MAPE
Gradient Boosting Regression  (train)        90.471361  0.115596
Gradient Boosting Regression   (test)        88.165451  0.115675
Gradient Boosting Regression    (cv, train)  97.162927  0.119539
Gradient Boosting Regression  2 (train)      82.204653  0.109468
Gradient Boosting Regression  2 (test)       84.508497  0.112759
Gradient Boosting Regression  2 (cv, train)  94.526100  0.116996


## Random Forest

In [119]:
from sklearn.ensemble import RandomForestRegressor
RF_reg = RandomForestRegressor(max_depth=2, random_state=0)
RF_reg.fit(x_train, y_train)

pred_RF_reg = RF_reg.predict(x_test)
pred_RF_reg

array([42.84312492, 65.95504711, 88.06836519, ..., 42.84312492,
       42.84312492, 88.06836519])

In [121]:
## Посчитаем RMSE и MAPE на тренирововчной и тестовой выборках.
MSE_train_RF = mean_squared_error(y_train,  RF_reg.predict(x_train))
MSE_test_RF = mean_squared_error(y_test, pred_RF_reg)
MAPE_train_RF = mean_absolute_percentage_error(y_train,  RF_reg.predict(x_train))
MAPE_test_RF = mean_absolute_percentage_error(y_test, pred_RF_reg)

# Посчитаем RMSE и MAPE с помощью кросс-валидации (чем меньше, тем лучше)
MSE_train_RF_cv  = -np.mean(cross_val_score(RF_reg, x_train, y_train, scoring='neg_mean_squared_error', cv = 5))
MSE_test_RF_cv = -np.mean(cross_val_score(RF_reg, x_test, y_test, scoring='neg_mean_squared_error', cv = 5))
MAPE_train_RF_cv  = -np.mean(cross_val_score(RF_reg, x_train, y_train, scoring='neg_mean_absolute_percentage_error', cv = 5))
MAPE_test_RF_cv = -np.mean(cross_val_score(RF_reg, x_test, y_test, scoring='neg_mean_absolute_percentage_error', cv = 5))

## Выводим все метрики до подбора пораметров
print(pd.DataFrame(data    = {'MSE': [
                              MSE_train_RF,
                              MSE_test_RF,
                              MSE_train_RF_cv
                              ],
                              'MAPE' : [
                              MAPE_train_RF,
                              MAPE_test_RF,
                              MAPE_train_RF_cv,
                              ]},
                   index   = ['Random Forest  (train)',
                              'Random Forest  (test)',
                              'Random Forest  (cv, train)',
                              ],
                   columns = ['MSE', 'MAPE']))

                                   MSE      MAPE
Random Forest  (train)      236.937307  0.182127
Random Forest  (test)       222.393247  0.177979
Random Forest  (cv, train)  237.512975  0.182579


In [123]:
from sklearn.model_selection import GridSearchCV

# подбор гиперпараметров
# перебираемые значения гиперпараметров
grid_space={'max_depth': [5, 10,15, 20, 25],
              'n_estimators':[100, 200, 300, 400],
              'max_features':[5,9, 12, 15],
           }

# Перебор гиперпараметров с помощью кросс-валидации
GSCV_RF_reg = GridSearchCV(estimator  = RF_reg,
                       param_grid     = grid_space,
                       scoring        = "neg_mean_squared_error",
                       cv             = 5)
model_GSCV_RF_reg = GSCV_RF_reg.fit(x_train, y_train)

# Достанем гиперпараметры, соответствующие лучшей модели
hyperparameters_RF_reg = GSCV_RF_reg.best_params_
print('Best hyperparameters are: '+str(hyperparameters_RF_reg))
print('Best score is: '+str(-model_GSCV_RF_reg.best_score_))

Best hyperparameters are: {'max_depth': 10, 'max_features': 5, 'n_estimators': 400}
Best score is: 100.82478317641467


In [125]:
print('Best hyperparameters are: '+str(hyperparameters_RF_reg))
print('Best score is: '+str(-model_GSCV_RF_reg.best_score_))

Best hyperparameters are: {'max_depth': 10, 'max_features': 5, 'n_estimators': 400}
Best score is: 100.82478317641467


In [127]:
# Обучим модель с лучшими гиперпараметрами
RF_reg2 = RandomForestRegressor(max_depth = hyperparameters_RF_reg['max_depth'],
                             max_features = hyperparameters_RF_reg['max_features'],
                             max_samples  = 500,
                             random_state = 777,
                             criterion    = 'squared_error',   ## это не RMSE, но вроде так и должно быть (немного путаюсь)
                             n_estimators = hyperparameters_RF_reg['n_estimators'])

RF_reg2.fit(x_train, y_train)

RandomForestRegressor(max_depth=10, max_features=5, max_samples=500,
                      n_estimators=400, random_state=777)

In [129]:
# Оценим точность модели с подобранными гиперпараметрами
MSE_train_RF2 = mean_squared_error(y_train,  RF_reg2.predict(x_train)) # обучающая выборка
MSE_test_RF2 = mean_squared_error(y_test,  RF_reg2.predict(x_test)) # тестовая выборка

MAPE_train_RF2 = mean_absolute_percentage_error(y_train,  RF_reg2.predict(x_train))
MAPE_test_RF2 = mean_absolute_percentage_error(y_test,  RF_reg2.predict(x_test))

# Сравним результаты кросс-валидации (чем меньше, тем лучше)
MSE_train_RF2_cv  = -np.mean(cross_val_score(RF_reg2, x_train, y_train, scoring='neg_mean_squared_error', cv = 5))
MSE_test_RF2_cv = -np.mean(cross_val_score(RF_reg2, x_test, y_test, scoring='neg_mean_squared_error', cv = 5))

MAPE_train_RF2_cv = -np.mean(cross_val_score(RF_reg2, x_train, y_train, scoring='neg_mean_absolute_percentage_error', cv = 5))
MAPE_test_RF2_cv = -np.mean(cross_val_score(RF_reg2, x_test, y_test, scoring='neg_mean_absolute_percentage_error', cv = 5))

## Выводим RMSE и MAPE метрики до подбора пораметров и после
print(pd.DataFrame(data    = {'MSE': [
                              MSE_train_RF,
                              MSE_test_RF,
                              MSE_train_RF_cv,
                              MSE_train_RF2,
                              MSE_test_RF2,
                              MSE_train_RF2_cv
                              ],
                              'MAPE' : [
                              MAPE_train_RF,
                              MAPE_test_RF,
                              MAPE_train_RF_cv,
                              MAPE_train_RF2,
                              MAPE_test_RF2,
                              MAPE_train_RF2_cv
                              ]},
                   index   = ['Random Forest   (train)',
                              'Random Forest   (test)',
                              'Random Forest   (cv, train)',
                              'Random Forest 2 (train)',
                              'Random Forest 2 (test)',
                              'Random Forest 2 (cv, train)'
                              ],
                   columns = ['MSE', 'MAPE']))

                                    MSE      MAPE
Random Forest   (train)      236.937307  0.182127
Random Forest   (test)       222.393247  0.177979
Random Forest   (cv, train)  237.512975  0.182579
Random Forest 2 (train)       94.484466  0.117640
Random Forest 2 (test)        97.985693  0.121220
Random Forest 2 (cv, train)  104.940158  0.123754


## Прогноз дохода для тестовой выборки

In [131]:
TEST = pd.read_excel('Турнир 1 test.xlsx')  
TEST.head()

,age,male,educ,weight,height,children,soil,serfdom,progress
0,17,0,0,59,1.50,2,1,0,0
1,23,1,0,73,1.66,6,0,0,0
2,19,1,0,76,1.82,2,0,0,0
3,20,0,0,77,1.68,6,1,0,0
4,20,1,0,76,1.65,5,1,1,1


In [133]:
# predict
predictions = GB_reg2.predict(TEST)

results = pd.DataFrame({'ATE': ATE_dml_standard, 'Target': predictions})
results.to_excel('Журба_Турнир.xlsx', index = False)

In [135]:
results

,ATE,Target
0,-3.941823,34.915151
1,-3.941823,77.998846
2,-3.941823,44.317938
3,-3.941823,57.375433
4,-3.941823,69.942711
...,...,...
89995,-3.941823,60.428645
89996,-3.941823,65.791281
89997,-3.941823,55.307752
89998,-3.941823,49.782941
